In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import time, timedelta
import random
from os import listdir
from os.path import isfile, join
random.seed(30)

In [2]:
df=pd.read_csv('distance_matrix.csv', sep=',',header=None)
file_index = 7
mypath = "./production and attraction python code"
# files = ["row1.csv",#0
# "row2.csv",#1
# "row3_employed_homebased_shopping.csv",#2
# "row3_unemployed_homebased_shopping.csv",#3
# "row4.csv",#4
# "row5.csv",#5
# "row6_homebound_employed_shoppers.csv",#6
# "row6_homebound_unemployed_shoppers.csv",#7
# "row7_afterwork_recreate.csv",#8
# "row7_coming_home_after_recreate.csv",#9
# "row7_comingfromlunch.csv",#10
# "row7_goingtolunch.csv",#11
# "row8_afterschool_extracurricular.csv",#12
# "row8_goinghome_extracurricular.csv",#13
# "row9_employed_homebound_shopping.csv",#14
# "row9_more_shopping_employed.csv",#15
# "row9_more_shopping_unemployed.csv",#16
# "row9_unemployed_homebound_shopping.csv"]#17
files = ['home_to_work',#0
            'home_to_k12',#1
            'home_to_college',#2
            'home_to_school',#3
            'home_to_SR_unemployed',#4
            'home_to_SR_employed',#5
            'work_to_home',#6
            'school_to_home',#7
            'SR_to_home_unemployed',#8
            'SR_to_home_employed',#9
            'work_to_lunch',#10
            'lunch_to_work',#11
            'work_to_SR',#12
            'work_SR_to_home',#13
            'school_to_SR',#14
            'school_SR_to_home',#15
            'SR_to_SR_unemployed',#16
            'SR_to_SR_employed',#17
            'SR_SR_to_home_unemployed',#18
            'SR_SR_to_home_employed']#19
#files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f[-4:] == ".csv"]

In [3]:
#start = ['06:30:00','06:30:00','18:00:00','10:00:00','16:00:00','15:00:00','11:00:00','19:00:00','16:00:00','17:00:00','12:30:00','11:30:00','15:00:00','17:00:00','10:00:00','20:00:00','20:00:00', '10:00:00']
#end = ['08:30:00','08:00:00','21:00:00','14:00:00','18:00:00','16:00:00','15:00:00','22:00:00','17:00:00','18:00:00','13:30:00','12:30:00','16:00:00','18:00:00','20:00:00','23:30:00','23:30:00','20:00:00']
start = ["06:30:00","07:30:00","08:30:00","09:30:00","18:00:00","10:00:00","16:00:00","15:00:00","11:00:00","19:00:00","16:00:00","17:00:00","12:30:00","11:30:00","15:00:00","17:00:00","10:00:00","11:00:00","20:00:00","20:00:00"]
end =   ["08:30:00","08:30:00","08:00:00","08:00:00","21:00:00","14:00:00","18:00:00","16:00:00","15:00:00","22:00:00","17:00:00","18:00:00","13:30:00","12:30:00","16:00:00","18:00:00","20:00:00","20:00:00","23:30:00","23:30:00"]


In [4]:
print(len(files))
print(len(start))
print(len(end))

20
20
20


In [5]:
DM = pd.DataFrame(df).to_numpy()
#distance matrix? with diagonals representing the average distance traveled to get to zone center
D = DM

In [6]:
# Gravity model - given A, we want A^* to be used so sum_i T_ij = A_j
# A_j is the total trip attraction at zone j
#P_i = total trip production at zone i
#T_ij is the trips produced at I and attracted to j
#F_ij is the calibration term for interchange ij also known as the friction factor or tracel time factor

def calculate_T(A, P, D):
    # invert and square for disutility
    F = np.power(D,-2)
    T = np.zeros_like(D)
    
    for i in range(T.shape[0]):
        for j in range(T.shape[1]):
            denom = F[i,:] @ A
            T[i,j] = P[i]*F[i,j]*A[j]/denom
    
    return T
    
def calculate_A_star(A, P, D):
    A_old = A.copy()
    C = np.ones_like(A)
    eps = 1e1
    n = A.shape[0]
    
    while (np.linalg.norm(C-A) > eps):
        T = calculate_T(A_old, P, D)
        C = T.sum(axis=0).reshape(n,1)
        
        for j in range(n):
            if C[j] != 0:
                A_old[j] = A[j]*A_old[j] / C[j]
        
    return A_old

In [7]:
def generate_figure(D, file_index):
    df1 = pd.read_csv(mypath + "/" + files[file_index] + ".csv", sep=',',header=None).fillna(0)
    header = df1.iloc[0]
    df1 = df1.iloc[1:79]
    df1.columns = header
    #df1 = df1.drop(labels = ['iii', 'TAZ (8a)', 'iv'],axis = 1)
    vec = df1
    #convert entire dataframe from strings to int64 types. Select P and A arrays by column.
    vec = vec.apply(pd.to_numeric)
    trip_arr = vec.to_numpy()
    P = trip_arr[:,1]
    A = trip_arr[:,2]
    dims = A.shape[0]
    A = A.reshape(dims, 1)
    P = P.reshape(dims, 1)
    A_star = calculate_A_star(A, P, D)
    T = calculate_T(A_star, P, D)
    #cumulative dist
    N = np.unravel_index(np.argsort(D.ravel()), np.shape(D))

    y = np.zeros(np.size(T))
    y_dens = np.zeros(np.size(T))
    x = np.zeros(np.size(T))
    for i in range(np.size(T)):
        if i == 0:
            y[i] = T[N[0][i], N[1][i]]
        else:
            y[i] = y[i-1] + T[N[0][i], N[1][i]]
            y_dens[i] = T[N[0][i], N[1][i]]
        x[i] = D[N[0][i], N[1][i]]
    
    #plots
    output_name = files[file_index][:-4]
    normalize = True
    plt.figure()
    if normalize:
        plt.plot(x,y/max(y))
    else:
        plt.plot(x,y)
    plt.xlabel('Distance(miles)')
    plt.ylabel('Probability d < Distance')
    plt.title("CDF of trip distance for " + output_name)
    plt.grid()
    plt.savefig("plots/" + output_name + "_cumdist.png")
    plt.clf()
    plt.close()
    
    plt.figure()
    if normalize:
        plt.plot(x,y_dens/np.sum(y_dens))
    else:
        plt.plot(x,y_dens)
    plt.xlabel('Distance(miles)')
    plt.ylabel('Probability Density')
    plt.title("PDF of trip distance for "+output_name)
    plt.grid()
    plt.savefig("plots/" + output_name + "_dendist.png")
    plt.clf()
    plt.close()

In [8]:
for i in range(len(files)):
    generate_figure(D,i)
    print("generated plots for iteration: " + str(i))
print(">>>>>>DONE<<<<<<<")

generated plots for iteration: 0
generated plots for iteration: 1
generated plots for iteration: 2
generated plots for iteration: 3
generated plots for iteration: 4
generated plots for iteration: 5
generated plots for iteration: 6
generated plots for iteration: 7
generated plots for iteration: 8
generated plots for iteration: 9
generated plots for iteration: 10
generated plots for iteration: 11
generated plots for iteration: 12
generated plots for iteration: 13
generated plots for iteration: 14
generated plots for iteration: 15
generated plots for iteration: 16
generated plots for iteration: 17
generated plots for iteration: 18
generated plots for iteration: 19
>>>>>>DONE<<<<<<<
